In [1]:
import math
import json
import itertools
import numpy as np
import requests
import pandas as pd
import time
import praw
from datetime import datetime, timedelta

In [2]:
USERNAME = 'mycoalring'
PASSWORD = 'Kan4Kan4Zhe4Ge!'
CLIENT_ID = 'OlAk1-JhpsqidA'
CLIENT_SECRET = 'FAxziWfQW5VVbgNDy1lco0758L9kxA'

reddit = praw.Reddit(
user_agent="Comment Extraction (by u/mycoalring)",
client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
username=USERNAME,
password=PASSWORD)

In [3]:
def get_request_uri(subreddit, after, before, size):
    uri = f'https://api.pushshift.io/reddit/search/submission?subreddit={subreddit}&after={after}&before={before}&size={size}'
    return uri

In [4]:
def make_request(uri, max_retries = 5):
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)
    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1)
            current_tries += 1
    return fire_away(uri)

# Get TSLA comments

In [5]:
def get_datetime_epoch(date_str):
    return int(datetime.strptime(date_str, '%Y-%m-%d').timestamp())

In [6]:
start_date = '2020-12-01'
end_date = '2020-12-10'

start_date_epoch = get_datetime_epoch(start_date)
end_date_epoch = get_datetime_epoch(end_date)
req_uri = get_request_uri('teslainvestorsclub', start_date_epoch, end_date_epoch,100)

In [7]:
start_date_epoch

1606809600

In [8]:
data = make_request(req_uri)['data']

In [9]:
data

[{'all_awardings': [],
  'allow_live_comments': False,
  'author': 'ianos0170',
  'author_flair_css_class': None,
  'author_flair_richtext': [],
  'author_flair_text': None,
  'author_flair_type': 'text',
  'author_fullname': 't2_13zzntis',
  'author_patreon_flair': False,
  'author_premium': False,
  'awarders': [],
  'can_mod_post': False,
  'contest_mode': False,
  'created_utc': 1606810259,
  'domain': 'self.teslainvestorsclub',
  'full_link': 'https://www.reddit.com/r/teslainvestorsclub/comments/k4gsca/is_tesla_auto_pilot_over_valued/',
  'gildings': {},
  'id': 'k4gsca',
  'is_crosspostable': False,
  'is_meta': False,
  'is_original_content': False,
  'is_reddit_media_domain': False,
  'is_robot_indexable': False,
  'is_self': True,
  'is_video': False,
  'link_flair_background_color': '',
  'link_flair_richtext': [],
  'link_flair_template_id': '489f2790-601e-11ea-bd6a-0e4917546149',
  'link_flair_text': 'Bear Thesis',
  'link_flair_text_color': 'dark',
  'link_flair_type': 'te

In [7]:
ids = []
for datapoint in data: 
    ids.append(datapoint['id'])

In [12]:
submission = reddit.submission(id=ids[3])

In [24]:
titles = []
upvote_ratios = []
scores = []

for _id in ids: 
    submission = reddit.submission(_id)
    titles.append(submission.title)
    upvote_ratios.append(submission.upvote_ratio)
    scores.append(submission.score)

In [27]:
df = pd.DataFrame({'titles': titles, 'upvote_ratios': upvote_ratios, 'scores': scores})

In [35]:
df.sort_values('scores', inplace=True, ascending=False)
df.head()

,titles,upvote_ratios,scores
45,"Goldman Sachs Upgrades Tesla (TSLA) to Buy, PT...",1.00,409
85,Tesla CEO Elon Musk has told friends and assoc...,0.95,333
9,Elon Musk calls for Tesla to focus on profits ...,0.98,277
22,Got my SEXY short shorts :),0.96,241
28,X-post All govt departments now required to bu...,0.98,232


In [46]:
df[(df['upvote_ratios'] > 0.5) & (df['scores'] > 10)]

,titles,upvote_ratios,scores
45,"Goldman Sachs Upgrades Tesla (TSLA) to Buy, PT...",1.00,409
85,Tesla CEO Elon Musk has told friends and assoc...,0.95,333
9,Elon Musk calls for Tesla to focus on profits ...,0.98,277
22,Got my SEXY short shorts :),0.96,241
28,X-post All govt departments now required to bu...,0.98,232
70,Swissquote Announces Partnership with Tesla to...,0.99,226
57,'Big Short' investor Michael Burry reveals he'...,0.94,186
3,VW CEO reveals ‘Mission T’ to ‘catch up with T...,0.97,171
58,Tesla Bear Jim Chanos Says He’d Tell Elon Musk...,0.98,167
33,Elon Musk is 'extremely confident' Tesla will ...,0.97,98


In [15]:
dir(submission)

['STR_FIELD',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_chunk',
 '_comments',
 '_comments_by_id',
 '_fetch',
 '_fetch_data',
 '_fetch_info',
 '_fetched',
 '_kind',
 '_reddit',
 '_reset_attributes',
 '_safely_add_arguments',
 '_url_parts',
 '_vote',
 'all_awardings',
 'allow_live_comments',
 'approved_at_utc',
 'approved_by',
 'archived',
 'author',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_template_id',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'banned_at_utc',
 'banned_by',
 'can